In [3]:
%config Completer.use_jedi=False
import torch
import numpy as np
import apex.amp as amp
import torchvision as tv
import matplotlib.pyplot as plt

import pytorch_tools as pt
from pytorch_tools.utils.misc import count_parameters

from src.dali_dataloader import DaliLoader

# Evaluate

In [4]:
INP = torch.ones(2, 3, 224, 224).cuda()
PRED = torch.ones(2, 1000).cuda()
LOSS = pt.losses.CrossEntropyLoss().cuda()
METRIC = [pt.metrics.Accuracy(), pt.metrics.Accuracy(5)]
@torch.no_grad()
def evaluate_model(model, size=224):
    """Eval model on ImageNet validation dataset"""
    BS = 25 if size >= 384 else 125
    val_loader = DaliLoader(False, bs=BS, sz=size, workers=12)
    model = model.eval().cuda()
    model = amp.initialize(model, opt_level='O1', verbosity=0)
    runner = pt.fit_wrapper.Runner(model, None, LOSS, metrics=METRIC, ) # callbacks=None
    _, (acc1, acc5) = runner.evaluate(val_loader)
    print(f"Acc@1: {acc1:.2f}. Acc@5: {acc5:.2f}")
    return acc1, acc5

In [12]:
out = pt.segmentation_models.Unet(
    encoder_weights=None,
    norm_layer="agn",
    decoder_channels=(256, 128, 64, 32, 32)
)(torch.ones(2, 3, 224, 224))

In [7]:
pt.models.resnet18(norm_layer="agn")# .cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): AGN(64, eps=1e-05, affine=True, activation=ACT.RELU)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): AGN(64, eps=1e-05, affine=True, activation=ACT.RELU)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): AGN(64, eps=1e-05, affine=True, activation=ACT.IDENTITY)
      (final_act): ReLU(inplace=True)
      (blurpool): Identity()
      (drop_connect): Identity()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): AGN(64, eps=1e-05, affine=True, activation=ACT.RELU)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (b

In [12]:
m1 = pt.models.resnet101(pretrained="imagenet_gn_ws")# .cuda()
m1 = pt.modules.conv_to_ws_conv(m1)
# m1.load_state_dict(torch.load("/home/zakirov/X-50-GN-WS.pth"))

In [ ]:
# R-50 GN + WS Acc@1: 76.33. Acc@5: 93.34 
# X-50-GN-WS Acc@1: 77.28. Acc@5: 93.61
# R-101 GN + WS Acc@1: 77.85. Acc@5: 93.90
# X-101-GN-WS Acc@1: 78.19. Acc@5: 93.98

In [13]:
evaluate_model(m1)

Epoch  1/1. validating:   4% 17/400 [00:08<03:07,  2.05it/s, Acc@1=25.118, Acc@5=44.600, Loss=4.5442]
Epoch  1/1. validating:   3% 12/400 [00:02<01:12,  5.39it/s, Acc@1=90.548, Acc@5=97.658, Loss=0.3915]

KeyboardInterrupt: 